In [1]:
import cv2
import os
import time
import numpy as np
import tensorflow as tf
import time
import collections
import PIL
from tensorflow.python.framework import graph_io
from tensorflow.keras.models import load_model
from tqdm import tqdm_notebook as tqdm
import tensorflow.contrib.tensorrt as trt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
import matplotlib.pyplot as plt
import tensorflow.contrib.tensorrt as trt

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
os.environ["TF_CPP_VMODULE"] = "segment=2,convert_graph=2,convert_nodes=2,trt_engine=2,trt_logger=2"

In [3]:
input_names = ['input_image', 'input_image_meta']

output_names = ["detections", "mrcnn_class", "mrcnn_bbox", "mrcnn_mask", "rois", "rpn_class", "rpn_bbox"]
output_names = ["output_" + name for name in output_names]

In [4]:
def get_frozen_graph(graph_file):
    """Read Frozen Graph file from disk."""
    with tf.gfile.GFile(graph_file, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    return graph_def

In [5]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14755807762922340991
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 14644791959986232049
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 393643163421037132
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5158019072
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16698919545797226796
physical_device_desc: "device: 0, name: Xavier, pci bus id: 0000:00:00.0, compute capability: 7.2"
]


In [6]:
# Create session and load graph
tf_config = tf.ConfigProto(log_device_placement=True)
tf_config.gpu_options.allow_growth = True
# tf_sess = tf.Session(config=tf_config)

# TRT

In [7]:
frozen_graph = get_frozen_graph('../logs/mask_rcnn_512_cocoperson_0396.pb')

In [8]:
for node in frozen_graph.node:
    node.device = "/device:XLA_GPU:0"

In [9]:
#with tf.device(DEVICE):
# tf.import_graph_def(frozen_graph, name='')

In [10]:
# frozen_graph = tf.get_default_graph().as_graph_def()

In [11]:
# tf.Graph.as_graph_def()

In [9]:
trt_graph = trt.create_inference_graph(
    input_graph_def=frozen_graph,
    outputs=output_names,
    is_dynamic_op=True,
    maximum_cached_engines=1,
    max_batch_size=1,
    max_workspace_size_bytes=1 << 30,
    precision_mode='FP16',
    minimum_segment_size=1,
    session_config=tf_config
)

INFO:tensorflow:Linked TensorRT version: (5, 1, 6)
INFO:tensorflow:Loaded TensorRT version: (5, 1, 6)
INFO:tensorflow:Running against TensorRT version 5.1.6


In [8]:
graph_io.write_graph(trt_graph, "../logs/",
                     "mask_rcnn_512_cocoperson_0396_trt.pb", as_text=False)

'../logs/mask_rcnn_512_cocoperson_0396_trt.pb'

In [23]:
cp = tf.ConfigProto(log_device_placement=True)

In [24]:
{n:getattr(cp, n) for n in dir(cp) if not n.startswith("_")}

{'ALLOW_SOFT_PLACEMENT_FIELD_NUMBER': 7,
 'ByteSize': <bound method _AddByteSizeMethod.<locals>.ByteSize of log_device_placement: true
 >,
 'CLUSTER_DEF_FIELD_NUMBER': 14,
 'Clear': <bound method _Clear of log_device_placement: true
 >,
 'ClearExtension': <bound method Message.ClearExtension of log_device_placement: true
 >,
 'ClearField': <bound method _AddClearFieldMethod.<locals>.ClearField of log_device_placement: true
 >,
 'CopyFrom': <bound method Message.CopyFrom of log_device_placement: true
 >,
 'DESCRIPTOR': <google.protobuf.descriptor.Descriptor at 0x7f4750dba8>,
 'DEVICE_COUNT_FIELD_NUMBER': 1,
 'DEVICE_FILTERS_FIELD_NUMBER': 4,
 'DeviceCountEntry': tensorflow.core.protobuf.config_pb2.DeviceCountEntry,
 'DiscardUnknownFields': <bound method _DiscardUnknownFields of log_device_placement: true
 >,
 'EXPERIMENTAL_FIELD_NUMBER': 16,
 'Experimental': tensorflow.core.protobuf.config_pb2.Experimental,
 'FindInitializationErrors': <bound method _AddIsInitializedMethod.<locals>.Find